In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/fr/job/R-34410?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Postuler pour Business Data Analyst II, Athlete Businesses

Rechercher des postesPasser la navigationRechercher des postesNIKE, INC. JOBSEmplois temporairesJoin The Talent CommunityNike au quotidienPrésentationAvantages sociauxMarquesPrésentationJordanConverseÉquipesPrésentationSoutien administratifInnovation de pointeAir Manufacturing et InnovationAéronautiqueCommunicationsService clientConceptionNumériqueInstallationsFinance et comptabilitéAffaires gouvernementales et publiquesRessources humainesDonnées et analyseService juridiqueFabrication et ingénierieMarketingMerchandisingPlanificationConfidentialitéAchatsCréation, développement et gestion de produitsService de gestion des magasinsMagasinsVentesImpact social et communautaireMarketing sportsPlanification stratégiqueChaîne d'approvisionnement, distribution et logistiqueDurabilitéTechnologieSitesPrésentationNike WHQSiège de Nike à New YorkEHQ : Hilversum, Pays-BasELC: Laakdal, BelgiumGreater China HQDiversité, égalité et inclusivité

In [21]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON not list.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

{
  "role": "Business Data Analyst II, Athlete Businesses",
  "experience": "2+ years of corporate development, business development, corporate strategy, investment banking, strategy consulting, finance, or related experience",
  "skills": [
    "Ability to complete quantitative and qualitative analysis and financial models",
    "Experience conducting and/or analyzing research related to consumers, industries, and competitors",
    "Comfort analyzing research related to consumers, industries, and competitors",
    "Proven ability to operate with a high degree of ambiguity and to work with a flexible schedule",
    "Logical problem structuring and strategic problem-solving skills",
    "Superior interpersonal skills and proven track record to work cross-functionally",
    "Excellent written, oral, and communication skills",
    "Ability to work on cross-functional teams and provide project management support",
    "Proficiency in standard business software applications (e.g., Keynote, 

In [22]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Business Data Analyst II, Athlete Businesses',
 'experience': '2+ years of corporate development, business development, corporate strategy, investment banking, strategy consulting, finance, or related experience',
 'skills': ['Ability to complete quantitative and qualitative analysis and financial models',
  'Experience conducting and/or analyzing research related to consumers, industries, and competitors',
  'Comfort analyzing research related to consumers, industries, and competitors',
  'Proven ability to operate with a high degree of ambiguity and to work with a flexible schedule',
  'Logical problem structuring and strategic problem-solving skills',
  'Superior interpersonal skills and proven track record to work cross-functionally',
  'Excellent written, oral, and communication skills',
  'Ability to work on cross-functional teams and provide project management support',
  'Proficiency in standard business software applications (e.g., Keynote, PowerPoint, Excel)'],
 'de

In [23]:
type(json_res)

dict

In [13]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [14]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [18]:
links = collection.query(query_texts=["experience in python", "expertise in React"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [24]:
job=json_res
job['skills']

['Ability to complete quantitative and qualitative analysis and financial models',
 'Experience conducting and/or analyzing research related to consumers, industries, and competitors',
 'Comfort analyzing research related to consumers, industries, and competitors',
 'Proven ability to operate with a high degree of ambiguity and to work with a flexible schedule',
 'Logical problem structuring and strategic problem-solving skills',
 'Superior interpersonal skills and proven track record to work cross-functionally',
 'Excellent written, oral, and communication skills',
 'Ability to work on cross-functional teams and provide project management support',
 'Proficiency in standard business software applications (e.g., Keynote, PowerPoint, Excel)']

In [25]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}]]

In [26]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Data-Driven Insights for Athlete Businesses with AtliQ

Dear Hiring Manager,

I came across the Business Data Analyst II, Athlete Businesses role at Nike and was impressed by the company's commitment to driving analytical work-streams that identify, evaluate, and catalyze action against critical priorities. As a Business Development Executive at AtliQ, I believe our expertise in AI and software consulting can help Nike unlock data-driven insights to inform key business decisions.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in developing and implementing data analytics solutions that drive business growth.

In particular, our expertise in machine learning and Python development can help Nike build robust data models and analytics tools that provide actionable insights. Our portfolio i